# Custom Tool and Profile Tutorial

This notebook walks through creating a simple custom tool, wiring it into a
profile, and running a mission via CLI or API.

Prereqs:
- Python 3.11+ environment with dependencies installed
- Run from the repo root (so paths like `configs/` and `src/` resolve)


## Step 1: Create a custom tool module

We will add a small tool that greets a user. The tool implements the
`ToolProtocol` interface and lives under `src/taskforce/infrastructure/tools/custom/`.


In [ ]:
from pathlib import Path

tool_dir = Path("src/taskforce/infrastructure/tools/custom")
tool_dir.mkdir(parents=True, exist_ok=True)
(tool_dir / "__init__.py").write_text("")

tool_code = '''from __future__ import annotations

from typing import Any

from taskforce.core.interfaces.tools import ApprovalRiskLevel, ToolProtocol


class HelloWorldTool(ToolProtocol):
    """Return a friendly greeting."""

    @property
    def name(self) -> str:
        return "hello_world"

    @property
    def description(self) -> str:
        return "Return a friendly greeting message."

    @property
    def parameters_schema(self) -> dict[str, Any]:
        return {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "Name to greet",
                }
            },
            "required": ["name"],
        }

    @property
    def requires_approval(self) -> bool:
        return False

    @property
    def approval_risk_level(self) -> ApprovalRiskLevel:
        return ApprovalRiskLevel.LOW

    def get_approval_preview(self, **kwargs: Any) -> str:
        return "Tool: hello_world"

    def validate_params(self, **kwargs: Any) -> tuple[bool, str | None]:
        if "name" not in kwargs or not kwargs["name"]:
            return False, "Missing required parameter: name"
        return True, None

    async def execute(self, **kwargs: Any) -> dict[str, Any]:
        name = kwargs.get("name", "there")
        return {"success": True, "output": f"Hello, {name}!"}

'''

(tool_dir / "hello_world_tool.py").write_text(tool_code)
print("Wrote hello_world_tool.py")


## Step 2: Create a profile that includes the tool

Profiles live under `configs/` and define infrastructure settings plus a
tool allowlist. This profile is intentionally minimal.


In [ ]:
from pathlib import Path

profile_path = Path("configs/hello_world.yaml")

yaml_text = '''profile: hello_world
specialist: generic
agent:
  enable_fast_path: true
  router:
    use_llm_classification: true
    max_follow_up_length: 100
persistence:
  type: file
  work_dir: .taskforce_hello_world
llm:
  config_path: configs/llm_config.yaml
  default_model: main
logging:
  level: DEBUG
  format: console
tools:
  - type: HelloWorldTool
    module: taskforce.infrastructure.tools.custom.hello_world_tool
    params: {}
mcp_servers: []
'''

profile_path.write_text(yaml_text)
print(f"Wrote {profile_path}")


## Step 3: Run the agent

CLI example:

```powershell
taskforce run mission "Use hello_world to greet Ada" --profile hello_world --lean
```

API example:

```bash
curl -X POST http://localhost:8000/api/v1/execute \
  -H "Content-Type: application/json" \
  -d '{"mission":"Use hello_world to greet Ada","profile":"hello_world","lean":true}'
```

Streaming example:

```bash
curl -X POST http://localhost:8000/api/v1/execute/stream \
  -H "Content-Type: application/json" \
  -d '{"mission":"Use hello_world to greet Ada","profile":"hello_world","lean":true}'
```
